Movie filters used: 
* runtime more than 80minutes
* relesaesed date after 1970
* production countries include USA
* language is engish

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import ast

In [2]:
c = pd.read_csv('credits.csv')
c.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [3]:
mm = pd.read_csv('movies_metadata.csv')
mm.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
mm.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
mm['original_language'].value_counts()

en       32269
fr        2438
it        1529
ja        1350
de        1080
         ...  
104.0        1
uz           1
lb           1
sm           1
rw           1
Name: original_language, Length: 92, dtype: int64

In [6]:
mm = mm[mm['original_language']=='en']

In [7]:
mm['release_date'] = pd.to_datetime(mm.release_date,errors='coerce')
mm['year'] = mm.release_date.dt.year
mm.year.value_counts().sort_index()

1878.0       1
1883.0       1
1890.0       2
1891.0       1
1892.0       2
          ... 
2015.0    1276
2016.0    1093
2017.0     411
2018.0       5
2020.0       1
Name: year, Length: 131, dtype: int64

In [8]:
mm[mm.year > 1975].shape

(24630, 25)

In [9]:
mm = mm[(mm.runtime >= 90) & (mm.year > 1975)]

In [10]:
mm['production_countries'].value_counts()

[{'iso_3166_1': 'US', 'name': 'United States of America'}]                                                                                                                                     8376
[]                                                                                                                                                                                             2197
[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]                                                                                                                                                916
[{'iso_3166_1': 'CA', 'name': 'Canada'}]                                                                                                                                                        420
[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]                                                                                      417
                    

In [11]:
mm.production_countries[0:2]

1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

In [12]:
def get_country_name(x):
    country = []
    st = ' '
    for i in x:
        if i.get('name')=='United States of America':
            country.append('USA')
    if country == []:
        return (' ')
    else:
        return (st.join(country))

In [13]:
mm['production_countries'] = mm['production_countries'].map(lambda x: ast.literal_eval(x))
mm['country'] = mm['production_countries'].apply(lambda x:get_country_name(x))

In [14]:
mm[mm.country == 'USA'].shape

(10629, 26)

In [15]:
mm.shape

(16318, 26)

In [16]:
mm = mm[mm.country == 'USA']

In [17]:
mm.shape,c.shape

((10629, 26), (45476, 3))

In [18]:
mm = mm[['title','id','genres']]

In [19]:
c['id'].describe()

count     45476.000000
mean     108345.997537
std      112443.796536
min           2.000000
25%       26443.250000
50%       60002.500000
75%      157302.000000
max      469172.000000
Name: id, dtype: float64

In [20]:
mm['id'].describe()

count      10629
unique     10625
top       109962
freq           2
Name: id, dtype: object

In [21]:
mm.isna().sum()

title     0
id        0
genres    0
dtype: int64

In [22]:
l = list(mm[mm['title'].notnull() == False].index)
mm.drop(l,inplace=True)

In [23]:
mm['id']=mm['id'].astype(int)
mm['id'].describe()

count     10629.000000
mean      84488.849751
std      109034.484990
min           5.000000
25%       12257.000000
50%       33295.000000
75%      103370.000000
max      463906.000000
Name: id, dtype: float64

In [24]:
c.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [25]:
mm.columns

Index(['title', 'id', 'genres'], dtype='object')

In [26]:
pd.set_option('display.max_colwidth',75)
data = pd.merge(mm,c,on='id')
data.head()

,title,id,genres,cast,crew
0,Jumanji,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id':...","[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a3...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', ..."
1,Grumpier Old Men,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]","[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a92514...","[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', '..."
2,Waiting to Exhale,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 1074...","[{'cast_id': 1, 'character': ""Savannah 'Vannah' Jackson"", 'credit_id': ...","[{'credit_id': '52fe44779251416c91011acb', 'department': 'Directing', '..."
3,Father of the Bride Part II,11862,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', 'credit_id': '52fe44959251...","[{'credit_id': '52fe44959251416c75039ed7', 'department': 'Sound', 'gend..."
4,Heat,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, ...","[{'cast_id': 25, 'character': 'Lt. Vincent Hanna', 'credit_id': '52fe42...","[{'credit_id': '52fe4292c3a36847f802916d', 'department': 'Directing', '..."


In [27]:
data['genres'][0]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [28]:
data['genres'] = data['genres'].map(lambda x: ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x: ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x: ast.literal_eval(x))

In [29]:
data['genres'][0]

[{'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 10751, 'name': 'Family'}]

In [30]:
def make_genrelist(x):
    gen = []
    st=' '
    for i in x:
        if i.get('name') == " Science Fiction":
            scifi = 'Sci-Fi'
            gen.append(scifi)
        elif i.get('name') == "TV Movie":
            continue
        else:
            gen.append(i.get('name'))
    if gen == []:
        return (' ')
    else:
        return (st.join(gen))

In [31]:
def get_actor1(x):
    cst = []
    for i in x:
            cst.append(i.get('name'))
    if cst == []:
        return (' ')
    else:
        return (cst[0])

In [32]:
def get_actor2(x):
    cst = []
    for i in x:
            cst.append(i.get('name'))
    if cst == [] or len(cst)<2:
        return (' ')
    else:
        return (cst[1])

In [33]:
def get_actor3(x):
    cst = []
    for i in x:
            cst.append(i.get('name'))
    if cst == [] or len(cst)<3:
        return (' ')
    else:
        return (cst[2])

In [34]:
def get_director(x):
    direc = []
    st = ' '
    for i in x:
        if i.get('job') == 'Director':
            direc.append(i.get('name'))
    if direc == []:
        return (' ')
    else:
        return (st.join(direc))

In [35]:
data['genres'] = data['genres'].apply(lambda x:make_genrelist(x))
data['actor_1_name'] = data['cast'].apply(lambda x:get_actor1(x))
data['actor_2_name'] = data['cast'].apply(lambda x:get_actor2(x))
data['actor_3_name'] = data['cast'].apply(lambda x:get_actor3(x))
data['director'] = data['crew'].apply(lambda x:get_director(x))

In [36]:
data.head(2)

,title,id,genres,cast,crew,actor_1_name,actor_2_name,actor_3_name,director
0,Jumanji,8844,Adventure Fantasy Family,"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a3...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', ...",Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
1,Grumpier Old Men,15602,Romance Comedy,"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a92514...","[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', '...",Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch


In [37]:
g = []
for i in data.genres:
    w = []
    w = i.split(' ')
    for j in w:
        if j not in g:
            g.append(j)
print(g)

['Adventure', 'Fantasy', 'Family', 'Romance', 'Comedy', 'Drama', 'Action', 'Crime', 'Thriller', 'History', 'Science', 'Fiction', 'Mystery', 'War', 'Music', 'Horror', 'Documentary', 'Foreign', 'Western', 'Animation', '']


In [38]:
data.shape

(10639, 9)

In [39]:
data.drop(['cast','crew'],axis=1,inplace=True)

In [40]:
data2 = data[['title','director','actor_1_name','actor_2_name','actor_3_name','genres']]

In [41]:
data2.isna().sum()

title           0
director        0
actor_1_name    0
actor_2_name    0
actor_3_name    0
genres          0
dtype: int64

In [42]:
data2.shape

(10639, 6)

In [43]:
data2 = data2.dropna()
data2.shape

(10639, 6)

In [44]:
data2 = data2.rename(columns = {'title':'movie_title','director':'director_name'})

In [45]:
data2.head(2)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,Jumanji,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family
1,Grumpier Old Men,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy


In [46]:
data2 = data2.apply(lambda x : x.str.lower())
data2.head()

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,jumanji,joe johnston,robin williams,jonathan hyde,kirsten dunst,adventure fantasy family
1,grumpier old men,howard deutch,walter matthau,jack lemmon,ann-margret,romance comedy
2,waiting to exhale,forest whitaker,whitney houston,angela bassett,loretta devine,comedy drama romance
3,father of the bride part ii,charles shyer,steve martin,diane keaton,martin short,comedy
4,heat,michael mann,al pacino,robert de niro,val kilmer,action crime drama thriller


In [47]:
data2['combined']=data2.director_name +' '+ data2.actor_1_name + ' ' + data2.actor_2_name + ' ' + data2.actor_3_name + ' ' + data2.genres
data2.head(1)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,jumanji,joe johnston,robin williams,jonathan hyde,kirsten dunst,adventure fantasy family,joe johnston robin williams jonathan hyde kirsten dunst adventure fanta...


In [48]:
data2.shape

(10639, 7)

In [49]:
old = pd.read_csv('data.csv')
old.tail(2)

,Unnamed: 0,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
3425,3425,shanghai calling,daniel hsia,alan ruck,daniel henney,eliza coupe,comedy drama romance,daniel hsia alan ruck daniel henney eliza coupe comedy drama romance
3426,3426,my date with drew,jon gunn,john august,brian herzlinger,jon gunn,documentary,jon gunn john august brian herzlinger jon gunn documentary


In [50]:
old.drop('Unnamed: 0',axis=1,inplace=True)
old.shape,old.head(1)

((3427, 7),
   movie_title  director_name actor_1_name      actor_2_name actor_3_name  \
 0      avatar  james cameron  cch pounder  joel david moore    wes studi   
 
                             genres  \
 0  action adventure fantasy sci-fi   
 
                                                                      combined  
 0  james cameron cch pounder joel david moore wes studi action adventure f...  )

In [51]:
new = old.append(data2)

In [52]:
new.shape

(14066, 7)

In [53]:
new.drop_duplicates(subset='movie_title',keep='last',inplace=True)
new.shape

(10795, 7)

In [54]:
new.reset_index(inplace = True)

In [55]:
new.drop('index',inplace=True,axis=1)
new

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,monsters vs. aliens,rob letterman,amy poehler,rainn wilson,stephen colbert,action adventure animation comedy family sci-fi,rob letterman amy poehler rainn wilson stephen colbert action adventure...
1,puss in boots,chris miller,salma hayek,constance marie,amy sedaris,action adventure animation comedy family fantasy,chris miller salma hayek constance marie amy sedaris action adventure a...
2,harry potter and the sorcerer's stone,chris columbus,daniel radcliffe,fiona shaw,verne troyer,adventure family fantasy,chris columbus daniel radcliffe fiona shaw verne troyer adventure famil...
3,die hard with a vengeance,john mctiernan,bruce willis,aldis hodge,kevin chamberlin,action adventure thriller,john mctiernan bruce willis aldis hodge kevin chamberlin action adventu...
4,the nutcracker in 3d,andrey konchalovskiy,shirley henderson,nathan lane,richard e. grant,action family fantasy musical,andrey konchalovskiy shirley henderson nathan lane richard e. grant act...
...,...,...,...,...,...,...,...
10790,an american vampire story,luis esteban\t,trevor lissauer,johnny venocur,adam west,comedy horror,luis esteban\t trevor lissauer johnny venocur adam west comedy horror
10791,fit to kill,andy sidaris,dona speir,roberta vasquez,bruce penhall,action romance thriller,andy sidaris dona speir roberta vasquez bruce penhall action romance th...
10792,cadet kelly,larry shaw,hilary duff,christy carlson romano,gary cole,comedy,larry shaw hilary duff christy carlson romano gary cole comedy
10793,robin hood,john irvin,patrick bergin,uma thurman,david morrissey,drama action romance,john irvin patrick bergin uma thurman david morrissey drama action romance


In [56]:
new.to_csv('new_data.csv',index=False)

In [57]:
g = []
for i in new.genres:
    w = []
    w = i.split(' ')
    for j in w:
        if j not in g:
            g.append(j)
print(g)

['action', 'adventure', 'animation', 'comedy', 'family', 'sci-fi', 'fantasy', 'thriller', 'musical', 'music', 'romance', 'war', 'crime', 'drama', 'horror', 'biography', 'history', 'mystery', 'sport', 'western', 'documentary', 'science', 'fiction', 'foreign', '']
